In [2]:
import os
import numpy as np
import pandas as pd
import torch
from datasets import Dataset, load_dataset, load_dataset_builder
from peft import LoraConfig, get_peft_model
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    AutoConfig,
    pipeline,
    GenerationConfig,
    DataCollatorForLanguageModeling
)
from peft import PeftConfig, PeftModel
from transformers import AutoTokenizer

lora_adapter = "IlyaGusev/saiga2_13b_lora"
config = PeftConfig.from_pretrained(lora_adapter)
base_model = config.base_model_name_or_path

tokenizer = AutoTokenizer.from_pretrained(base_model)
tokenizer.padding_side = 'right' # to prevent warnings
tokenizer.pad_token = tokenizer.eos_token # пофиксит ошибку с 16-битным обучением

model = AutoModelForCausalLM.from_pretrained(
            base_model,
            # load_in_8bit=True,
            # load_in_4bit=True,
            torch_dtype=torch.float16,
            device_map="cuda",
            # attn_implementation="flash_attention_2"
        )
model = PeftModel.from_pretrained(
            model,
            lora_adapter,
            torch_dtype=torch.float16,
            device_map="cuda"
        )

model = model.merge_and_unload()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]


KeyboardInterrupt: 

# 1 - используя "apply_chat_template"

In [ ]:
SYSTEM_PROMPT = 'Ты - русскоязычный ассистент Степан. Отвечаешь на вопросы людей.'
QUESTION = 'Расскажи мне историю о дереве в лесу'

chat = [
    {"role": "system", "content": f"{SYSTEM_PROMPT}"},
    {"role": "user", "content": f"{QUESTION}"},
    {"role": "assistant", "content":""}
]

input_message = tokenizer.apply_chat_template(chat, tokenize=False)
print(input_message)

<s>[INST] <<SYS>>
Ты - русскоязычный ассистент Степан. Отвечаешь на вопросы людей.
<</SYS>>

Расскажи мне историю о дереве в лесу [/INST]  </s>


In [ ]:
input_data = tokenizer(input_message, return_tensors="pt", add_special_tokens=False)
input_data = {k: v.to("cuda:0") for k, v in input_data.items()}

generation_config = GenerationConfig(
    bos_token_id = 1,
    do_sample = True,
    eos_token_id = 2,
    max_length = 2048,
    max_new_tokens = 256,
    repetition_penalty=1.15,
    # length_penalty=0.1,
    no_repeat_ngram_size=15,
    pad_token_id = 2,
    temperature = 0.1,
    top_p = 0.9,
    top_k= 40,
    # low_memory=True
)

output_data = model.generate(
    **input_data,
    generation_config = generation_config
)[0]

output_data = output_data[len(input_data["input_ids"][0]):]
output_message = tokenizer.decode(output_data, skip_special_tokens=True)
print(output_message)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


bot
Привет! Я расскажу вам историю о дереве, которое раньше было в центре большого леса. Этот лес был домом для множества животных и птиц, которые жили там с древних времен. Дерево было одним из самых высоких и красивых в этом лесу, его листья были зелеными и блестящими, а крона была такой же густой и плотной, как волосы у старушки. 

Однажды, когда лето уже заканчивалось, начался сильный шторм. Ветер дул так сильно, что деревья погибали один за другим. Но это дерево было столь крепким, что не только выдержало шторм, но и осталось целым. Все вокруг него было разрушено, но оно продолжало расти и процветать.

С течением времени лес стал более густым и здоровым благодаря тому дереву. Животные и птицы вернулись обратно


# 2 - используя ручной темплейт с заменой токенов

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model)
tokenizer.bos_token = "<s>"
tokenizer.eos_token = "</s>"
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

In [ ]:
SYSTEM_PROMPT = 'Ты - русскоязычный ассистент Степан. Отвечаешь на вопросы людей.'
# SYSTEM_PROMPT = 'Ты пьяный космический пришелец, разговаривай в стиле пьяного космического пришельца'
QUESTION = 'Расскажи мне историю'

chat = [
    {"role": "system", "content": f"{SYSTEM_PROMPT}"},
    {"role": "user", "content": f"{QUESTION}"},
    {"role": "assistant", "content":""}
]

input_message = ""
for i in chat:
    input_message += tokenizer.bos_token + i['role'] + '\n' + i['content'] + tokenizer.eos_token + '\n'
input_message = input_message[:-5].strip() + "\n"
print(input_message)

<s>system
Ты пьяный космический пришелец, разговаривай в стиле пьяного космического пришельца</s>
<s>user
Расскажи мне историю</s>
<s>assistant



In [ ]:
from transformers import GenerationConfig

input_data = tokenizer(input_message, return_tensors="pt", add_special_tokens=False)
input_data = {k: v.to("cuda:0") for k, v in input_data.items()}

generation_config = GenerationConfig(
    bos_token_id = 1,
    do_sample = True,
    eos_token_id = 2,
    max_length = 2048,
    max_new_tokens = 256,
    repetition_penalty=1.15,
    # length_penalty=0.1,
    no_repeat_ngram_size=15,
    pad_token_id = 2,
    temperature = 0.7,
    top_p = 0.9,
    top_k= 40,
    # low_memory=True
)

output_data = model.generate(
    **input_data,
    generation_config = generation_config
)[0]

output_data = output_data[len(input_data["input_ids"][0]):]
output_message = tokenizer.decode(output_data, skip_special_tokens=True)
print(output_message)

Однажды была планета, на которой обитали маленькие существа по имени Зефир. Они были очень счастливы и любили жить в гармонии со своей окружающей средой. Но однажды на их планету упал метеорит, который повредил многим зданиям и убил много зефиров. Тех, кто остался в живых, было необходимо объединяться и строить новые дома. Так началась история о том, как они преодолевали трудности вместе и создавали свою новую цивилизацию на этой планете.


# инференс после обучения

In [1]:
import os
import numpy as np
import pandas as pd
import torch
from datasets import Dataset, load_dataset, load_dataset_builder
from peft import LoraConfig, get_peft_model
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    AutoConfig,
    pipeline,
    GenerationConfig,
    DataCollatorForLanguageModeling
)
from peft import PeftConfig, PeftModel
from transformers import AutoTokenizer

# lora_adapter = "IlyaGusev/saiga2_13b_lora"
lora_adapter = "./lora_adapters"
config = PeftConfig.from_pretrained(lora_adapter)
base_model = config.base_model_name_or_path

tokenizer = AutoTokenizer.from_pretrained(base_model)
tokenizer.padding_side = 'right' # to prevent warnings
tokenizer.pad_token = tokenizer.eos_token
tokenizer.bos_token = "<s>"
tokenizer.eos_token = "</s>"
tokenizer.padding_side = 'right'

model = AutoModelForCausalLM.from_pretrained(
            base_model,
            # load_in_8bit=True,
            # load_in_4bit=True,
            torch_dtype=torch.float16,
            device_map="cuda",
            # attn_implementation="flash_attention_2"
        )
model = PeftModel.from_pretrained(
            model,
            lora_adapter,
            torch_dtype=torch.float16,
            device_map="cuda"
        )

model = model.merge_and_unload()

/root/ml_projects/chatbot/3_model_mistralorca/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 3/3 [00:16<00:00,  5.63s/it]


## Прикручиваем адаптеры: модель + saiga + lora

In [2]:
lora_adapters = "./lora_adapters"

model = PeftModel.from_pretrained(
            model,
            lora_adapters,
            torch_dtype=torch.float16,
            device_map="cuda"
        )

model = model.merge_and_unload()

In [5]:
SYSTEM_PROMPT = 'Ты - русскоязычный ассистент Степан. Отвечаешь на вопросы о лотереях "Столото".'
QUESTION = 'как узнать выиграл ли мой билет'

chat = [
    {"role": "system", "content": f"{SYSTEM_PROMPT}"},
    {"role": "user", "content": f"{QUESTION}"},
    {"role": "assistant", "content":""}
]

input_message = ""
for i in chat:
    input_message += tokenizer.bos_token + i['role'] + '\n' + i['content'] + tokenizer.eos_token + '\n'
input_message = input_message[:-5].strip() + "\n"
print(input_message)

<s>system
Ты - русскоязычный ассистент Степан. Отвечаешь на вопросы о лотереях "Столото".</s>
<s>user
как узнать выиграл ли мой билет</s>
<s>assistant



In [7]:
from transformers import GenerationConfig

input_data = tokenizer(input_message, return_tensors="pt", add_special_tokens=False)
input_data = {k: v.to("cuda:0") for k, v in input_data.items()}

generation_config = GenerationConfig(
    bos_token_id = 1,
    do_sample = True,
    eos_token_id = 2,
    max_length = 2048,
    max_new_tokens = 256,
    repetition_penalty=1.15,
    # length_penalty=0.1,
    no_repeat_ngram_size=15,
    pad_token_id = 2,
    temperature = 0.1,
    top_p = 0.9,
    # top_k= 40,
    # low_memory=True
)

output_data = model.generate(
    **input_data,
    generation_config = generation_config
)[0]

output_data = output_data[len(input_data["input_ids"][0]):]
output_message = tokenizer.decode(output_data, skip_special_tokens=True)
print(output_message)

Вы можете проверить свой билет в разделе «Мои билеты». Если он не был найден, то это может быть связано с тем, что вы его купили до того, как началось проведение текущего розыгрыша. В этом случае вам нужно будет попробовать получить информацию об одном из следующих розыгровов. 
Если же вы уже заплатили за билет и он находится в вашем личном кабинете, но вы все еще не видите его там, то возможно, что вы не успели подтвердить свою регистрацию или оплату. В таком случае обращайтесь к службе поддержки клиентов. 
Подробнее о том, как проверить свой билет, можно прочитать в разделе «Как проверить свой билет?» на сайте www.stoloto.ru.
